# VAE attempt 3
Based off of Alexander Van de Kleut's work. See [this post](https://avandekleut.github.io/vae/) for some theory and explination.

In [82]:
import torch; torch.manual_seed(0)
import json
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.distributions
import numpy as np
import matplotlib.pyplot as plt;
import time

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [83]:
### Paths ###

parent_folder = 'C:/Users/Aidan/Documents/Winter_2023/BE177B/Code/'
art_json = parent_folder + 'wonglab-capstone/Datateam/Artificial_imset/artificial_kymographs.json'
exp_json = parent_folder + 'wonglab-capstone/Datateam/imset1/experimental_kymograph.json'

class ArtKymoDataset(Dataset):
    def __init__(self):
        #data loading
        with open(art_json, 'r') as f:
            kymos = np.asarray(json.loads(f.read())["kymoset"])
        kymos = kymos.astype('float32')
        kymos = torch.from_numpy(kymos)
        # self.x = kymos[:,None,:,:]
        self.x = kymos
        self.x = self.x[:,None,:,:]
        self.n_samples = kymos.shape[0]

    def __getitem__(self, index):
        return self.x[index,:,:]

    def __len__(self):
        return self.n_samples

### Autoencoder architecture, notice this is **NOT** a VAE

In [104]:
class Encoder(nn.Module):
    def __init__(self, latent_dims):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(1200, 512) # input flattened kymograph
        self.linear2 = nn.Linear(512, latent_dims) # squishes into latent space
    
    def forward(self, x):
        x = torch.flatten(x, start_dim = 1) #may have to change start dim if not along right axis, I guessed randomly
        x = F.relu(self.linear2(F.relu(self.linear1(x))))
        return x

class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 512)
        self.linear2 = nn.Linear(512, 1200)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = torch.sigmoid(self.linear2(z))
        return z.reshape((-1, 1, 20, 60))

class Autoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(Autoencoder, self).__init__()
        self.encoder = Encoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

#### Pretty good training loop, can probably repurpose for VAE as well

In [105]:
def train(autoencoder, data, epochs=5):
    opt = torch.optim.Adam(autoencoder.parameters())
    for epoch in range(epochs):
        startTime = time.perf_counter()
        overall_loss = 0
        for x in data:
            x = x.to(device) # GPU
            opt.zero_grad()
            x_hat = autoencoder(x)
            loss = ((x - x_hat)**2).sum()   
            loss.backward()
            overall_loss += loss.item()
            opt.step()
        timediff = (time.perf_counter()-startTime)
        print("\tEpoch {} complete \tTotal Loss: {} \tTraining time: {} ".format(epoch+1, loss, timediff))
    return autoencoder

In [106]:
dataset = ArtKymoDataset()
latent_dims = 10
autoencoder = Autoencoder(latent_dims).to(device)
autoencoder = train(autoencoder, dataset)

	Epoch 1 complete 	Total Loss: 155038.65625 	Training time: 7.156905900000311 
	Epoch 2 complete 	Total Loss: 155038.65625 	Training time: 7.805396699999619 
	Epoch 3 complete 	Total Loss: 155038.65625 	Training time: 10.862655699999777 
	Epoch 4 complete 	Total Loss: 155038.65625 	Training time: 16.711969599999975 
	Epoch 5 complete 	Total Loss: 155038.65625 	Training time: 13.323557300000175 
